![](./resources/System_v1_training_header.png)

This notebook contains a demonstration on how to train custom crop type models based on your own reference data and how to apply the resulting model to generate a custom crop type map.

# Content

- [Before you start](#before-you-start)
- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Check public in-situ reference data](#2.-Check-public-in-situ-reference-data)
- [3. Prepare own reference data](#3.-Prepare-own-reference-data)
- [4. Extract required model inputs](#4.-Extract-required-model-inputs)
- [5. Train custom classification model](#5.-Train-custom-classification-model)
- [6. Generate a map](#6.-Generate-a-map)

# Before you start

In order to run this notebook, you need to create an account on:

- The Copernicus Data Space Ecosystem (CDSE)
--> by completing the form [HERE](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=eRKGqDvoYI0)

- VITO's Terrascope platform
--> by completing the form [HERE](https://sso.terrascope.be/auth/realms/terrascope/login-actions/registration?client_id=drupal-terrascope&tab_id=irBzckp2aDo)

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from worldcereal.utils.map import get_ui_map

RDM_API = "https://ewoc-rdm-api.iiasa.ac.at"

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [4]:
m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [5]:
# retrieve bounding box from drawn rectangle
from worldcereal.utils.map import get_bbox_from_draw

spatial_extent, bbox, poly = get_bbox_from_draw(dc)

Your area of interest: (4.91206, 51.17841, 4.974569, 51.210898)
Area of processing extent: 16.64 km²


# 2. Extract public in situ reference data

Here we query existing reference data that have already been processed by WorldCereal and are ready to use.
We filter for croptype labels by default, intersecting with a buffer around the bbox.

In [22]:
from utils import query_worldcereal_samples

public_df = query_worldcereal_samples(poly, buffer=50000, filter_cropland=False)

ImportError: attempted relative import with no known parent package

# 4. Extract required model inputs

Here we prepare presto features for each sample by using a model pretrained on WorldCereal data.

In [ ]:
from utils import get_inputs_outputs

encodings, targets = get_inputs_outputs(public_df, task_type = "cropland")

# 5. Train custom classification model
We train a catboost model for the selected crop types.

In [ ]:
from utils import train_classifier

custom_model, report = train_classifier(encodings, targets)

In [ ]:
# Print the classification report
print(report)

# 6. Deploy custom model

Once trained, we have to upload our model to the cloud so it can be used for inference.


In [ ]:
from utils import deploy_model

model_url = deploy_model(custom_model, pattern="demo")

# 7. Generate a map

Using our custom model, we generate a map for our region of interest...

In [ ]:
from worldcereal.job import WorldCerealProduct, generate_map, CropTypeParameters
from openeo_gfmap import TemporalContext

# Set temporal range to generate product
temporal_extent = TemporalContext(
    start_date="2021-11-01",
    end_date="2022-10-31",
)

# Initializes default parameters
parameters = CropTypeParameters()

# Change the URL to the classification model
parameters.classifier_parameters.classifier_url = model_url

# Launch the job
job_results = generate_map(
    spatial_extent,
    temporal_extent,
    output_path="./cropmap.tif",
    product_type=WorldCerealProduct.CROPTYPE,
    croptype_parameters=parameters,
    out_format="GTiff",
)